In [1]:
# This cell has a bit of initial setup. You can click the triangle to the left to expand it.
# Click the "Run" button immediately above the notebook in order to execute the contents of any cell
# WARNING: Each cell in the notebook relies upon results generated by previous cells
#   The most common problem beginners have is to execute a cell before all its predecessors
#   If you do this, you can restart the kernel (see the "Kernel" menu above) and start over
%matplotlib inline
import matplotlib.pyplot as plt

# The first step is to be able to bring things in from different directories
import sys 
import os

sys.path.insert(0, os.path.abspath('../lib'))

#from util import log_progress

import numpy as np
import HARK 
from time import clock
from copy import deepcopy
mystr = lambda number : "{:.4f}".format(number)
from HARK.utilities import plotFuncs

In [2]:
# Define a parameter dictionary with baseline parameter values

# Set the baseline parameter values 
PermGroFac = 1.02
Rfree      = 1.04
DiscFac    = 0.96
CRRA       = 2.00
UnempPrb   = 0.005
IncUnemp   = 0.0
PermShkStd = 0.1
TranShkStd = 0.1
# Import default parameter values
import HARK.ConsumptionSaving.ConsumerParameters as Params 

# Make a dictionary containing all parameters needed to solve the model
base_params = Params.init_idiosyncratic_shocks

# Set the parameters for the baseline results in the paper
# using the variable values defined in the cell above
base_params['PermGroFac'] = [PermGroFac]   # Permanent income growth factor
base_params['Rfree']      = Rfree          # Interest factor on assets
base_params['DiscFac']    = DiscFac        # Time Preference Factor
base_params['CRRA']       = CRRA           # Coefficient of relative risk aversion
base_params['UnempPrb']   = UnempPrb       # Probability of unemployment (e.g. Probability of Zero Income in the paper)
base_params['IncUnemp']   = IncUnemp       # Induces natural borrowing constraint
base_params['PermShkStd'] = [PermShkStd]   # Standard deviation of log permanent income shocks
base_params['TranShkStd'] = [TranShkStd]   # Standard deviation of log transitory income shocks

# Some technical settings that are not interesting for our purposes
base_params['LivPrb']       = [1.0]   # 100 percent probability of living to next period
base_params['CubicBool']    = True    # Use cubic spline interpolation
base_params['T_cycle']      = 1       # No 'seasonal' cycles
base_params['BoroCnstArt']  = None    # No artificial borrowing constraint



In [3]:
from HARK.ConsumptionSaving.ConsIndShockModel import IndShockConsumerType
baseEx_inf = IndShockConsumerType(cycles=0,**base_params)

base_params['PermGroFac']=[1.03]
baseEx_infg = IndShockConsumerType(cycles=0,**base_params)

baseEx_inf.solve()
baseEx_inf.unpackcFunc()

baseEx_infg.solve()
baseEx_infg.unpackcFunc()


In [4]:
import HARK.ConsumptionSaving.ConsIndShockModel as Model        # The consumption-saving micro model
import HARK.ConsumptionSaving.ConsumerParameters as Params    # Parameters for the consumer type
from HARK.utilities import plotFuncsDer, plotFuncs              # Some tools
from time import time
mystr = lambda number : "{:.4f}".format(number)
do_simulation = True
import numpy as np
import matplotlib.pyplot as plt

In [5]:
import math
from scipy.optimize import fsolve

table2 = np.zeros((7,8))
DeltaHW = (1/(1 - 1.03/Rfree)) - (1/(1 - 1.02/Rfree)) #Human wealth difference between two growth rates

for i in range(8):
    table2[i-1,0] = 0.4*i                        #Different Gross wealth ratios
    table2[i-1,4] = baseEx_inf.cFunc[0](0.4*i)   #consumption under 2% permanent income growth rate
    table2[i-1,5] = baseEx_infg.cFunc[0](0.4*i)   #consumption under 3% permanent income growth rate
    table2[i-1,6] = (table2[i-1,5]-table2[i-1,4])/DeltaHW  #MPC out of human wealth
    def func(x):
        return table2[i-1,5]-table2[i-1,4]-(1-(x**(-1)*((x*DiscFac)**(1/CRRA))))*((1/(1-(1.03/x)))-(1/(1-(1.02/x))))
    table2[i-1,7] = fsolve(func, 1.1) -1         # Implied discount rate of future income
    


/usr/local/anaconda3/lib/python3.7/site-packages/scipy/optimize/minpack.py:162: RuntimeWarning: The iteration is not making good progress, as measured by the 
  improvement from the last ten iterations.
  warnings.warn(msg, RuntimeWarning)


In [6]:
# Define a parameter dictionary with baseline parameter values

# Set the baseline parameter values 
PermGroFac = 1.02
Rfree      = 1.04
DiscFac    = 0.96
CRRA       = 2.00
UnempPrb   = 0.005
IncUnemp   = 0.0
PermShkStd = 0.1
TranShkStd = 0.1
# Import default parameter values
import HARK.ConsumptionSaving.ConsumerParameters as Params 

# Make a dictionary containing all parameters needed to solve the model
base_params = Params.init_idiosyncratic_shocks

# Set the parameters for the baseline results in the paper
# using the variable values defined in the cell above
base_params['PermGroFac'] = [PermGroFac]   # Permanent income growth factor
base_params['Rfree']      = Rfree          # Interest factor on assets
base_params['DiscFac']    = DiscFac        # Time Preference Factor
base_params['CRRA']       = CRRA           # Coefficient of relative risk aversion
base_params['UnempPrb']   = UnempPrb       # Probability of unemployment (e.g. Probability of Zero Income in the paper)
base_params['IncUnemp']   = IncUnemp       # Induces natural borrowing constraint
base_params['PermShkStd'] = [PermShkStd]   # Standard deviation of log permanent income shocks
base_params['TranShkStd'] = [TranShkStd]   # Standard deviation of log transitory income shocks

# Some technical settings that are not interesting for our purposes
base_params['LivPrb']       = [1.0]   # 100 percent probability of living to next period
base_params['CubicBool']    = True    # Use cubic spline interpolation
base_params['T_cycle']      = 1       # No 'seasonal' cycles
base_params['BoroCnstArt']  = None    # No artificial borrowing constraint



In [7]:
from HARK.ConsumptionSaving.ConsIndShockModel import PerfForesightConsumerType

baseEx_inf_PF = PerfForesightConsumerType(cycles=0,**base_params)

base_params['PermGroFac']=[1.03]
baseEx_inf_PFg = PerfForesightConsumerType(cycles=0,**base_params)

baseEx_inf_PF.solve()
baseEx_inf_PF.unpackcFunc()

baseEx_inf_PFg.solve()
baseEx_inf_PFg.unpackcFunc()



In [8]:
for i in range(8):
    table2[i-1,1] = baseEx_inf_PF.cFunc[0](0.4*i)           #Consumption in certainty model when permanent income growth rate is 2%
    table2[i-1,2] = baseEx_inf_PFg.cFunc[0](0.4*i)          #Consumption in certainty model when permanent income growth rate is 3%
    table2[i-1,3] = (table2[i-1,2] - table2[i-1,1])/DeltaHW #MPC out of human wealth in certainty model

In [9]:
import pandas as pd



# Data frame of the results we calculated
table = pd.DataFrame(table2)
table.columns = ['Gross wealth ratio', 'Consumption when g=2%', 'Consumption when g=3%', 'MPC out of human wealth','Consumption when g=2%', 'Consumption when g=3%', 'MPC out of human wealth', 'Implied Discount Rate of Future Income'] # add names for columns
table

,Gross wealth ratio,Consumption when g=2%,Consumption when g=3%,MPC out of human wealth,Consumption when g=2%,Consumption when g=3%,MPC out of human wealth,Implied Discount Rate of Future Income
0,0.4,2.016475,4.056489,0.039231,0.370145,0.370261,0.000002,77.197782
1,0.8,2.032168,4.072182,0.039231,0.713296,0.715560,0.000044,3.060224
2,1.2,2.047860,4.087874,0.039231,0.930450,0.946826,0.000315,0.415653
3,1.6,2.063553,4.103567,0.039231,1.024227,1.056702,0.000625,0.238690
4,2.0,2.079245,4.119259,0.039231,1.081466,1.127390,0.000883,0.186045
5,2.4,2.094938,4.134951,0.039231,1.123891,1.181877,0.001115,0.159205
6,2.8,2.110630,4.150644,0.039231,1.158950,1.227657,0.001321,0.142994
